<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Calculation Methods </span> </p

In [0]:
#!import "./Extensions"

# Get Previous Identities

In [0]:
public static Dictionary<AocStep,IEnumerable<AocStep>> GetPreviousIdentities(IEnumerable<AocStep> aocSteps)
{
    var bopNovelties = aocSteps.Where(id => id.AocType == AocTypes.BOP).Select(id => id.Novelty);
    var previousStep = (new string[]{Novelties.N,Novelties.I,Novelties.C})
                                        .ToDictionary(n => n, n => bopNovelties.Contains(n) ? new AocStep(AocTypes.BOP,n) : null);
    return aocSteps.Where(id =>  id.AocType != AocTypes.BOP)
                   .ToDictionary(x => x, 
                                 x => {var isFirstCombined = x.Novelty == Novelties.C && (previousStep[Novelties.C] == null || previousStep[Novelties.C].AocType == AocTypes.BOP);
                                       var ret = isFirstCombined
                                                  ? previousStep.Where(kvp => kvp.Value != null).Select(kvp => kvp.Value).ToArray() 
                                                  : previousStep[x.Novelty].RepeatOnce();
                                       previousStep[x.Novelty] = new AocStep(x.AocType, x.Novelty);
                                       return ret;});
}

# Get Reference AoC Step for calculated steps

In [0]:
public static AocStep GetReferenceAocStepForCalculated(this IEnumerable<AocStep> identities, Dictionary<AocStep, AocConfiguration> aocConfigurationByAocStep, AocStep identityAocStep)
{
    return identities.LastOrDefault(aocStep => aocConfigurationByAocStep[aocStep].DataType != DataType.Calculated
                                            && aocConfigurationByAocStep[aocStep].DataType != DataType.CalculatedTelescopic
                                            && aocConfigurationByAocStep[aocStep].Order < aocConfigurationByAocStep[identityAocStep].Order
                                            && aocStep.Novelty == identityAocStep.Novelty) 
        ?? new AocStep(default, default);
}

# Discount and Cumulate

Calculation method called from the Import Scopes. Expected inputs are nominal values and monthly discounting rate arrays and the period type which controls the details of the formula used. The monthly discounting rates can correspond to an empty array, this is the indication that this method should perform only the cumulation of the nominal withouth discounting. This is achieve by setting the monthly discounting to 1. 

In [0]:
public static double[] ComputeDiscountAndCumulate(this double[] nominalValues, double[] monthlyDiscounting, PeriodType periodType) 
{ 
    if(nominalValues == null) return Enumerable.Empty<double>().ToArray();
    if(!monthlyDiscounting.Any()) monthlyDiscounting = new double[]{1d}; //Empty discounting array triggers Cumulation. 

    var ret = new double[nominalValues.Length];
    
    if(periodType == PeriodType.BeginningOfPeriod)
    {
        for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = nominalValues[i] + ret.GetValidElement(i + 1) * monthlyDiscounting.GetValidElement(i/12);
        return ret;
    }
    
    for (var i = nominalValues.Length - 1; i >= 0; i--)
                ret[i] = ( nominalValues[i] + ret.GetValidElement(i + 1) ) * monthlyDiscounting.GetValidElement(i/12);
    
    return ret;
}

In [0]:
public static double[] ComputeDiscountAndCumulateWithCreditDefaultRisk(this double[] nominalValues, double[] monthlyDiscounting, double nonPerformanceRiskRate) //NonPerformanceRiskRate consider to be constant in time. Refinement would it be an array that takes as input tau/t.
{ 
    if(!monthlyDiscounting.Any()) monthlyDiscounting = new double[]{1d}; //Empty discounting array triggers Cumulation.
    return Enumerable.Range(0, nominalValues.Length)
                     .Select( t => Enumerable.Range(t, nominalValues.Length-t)
                                             .Select( tau => nominalValues[tau] * Math.Pow(monthlyDiscounting.GetValidElement(t/12), tau-t+1) * (Math.Exp(-nonPerformanceRiskRate*(tau-t)) - 1) )
                                             .Sum() )
                     .ToArray();
}

In [0]:
public static IDataCube<RawVariable> ComputeDiscountAndCumulate ( this IDataCube<RawVariable> nominalRawVariables, double[] yearlyDiscountRates, AmountType[] amountTypes ) 
{
    if(nominalRawVariables == null) return Enumerable.Empty<RawVariable>().ToDataCube();
    var periodTypeByAmountType = amountTypes.ToDictionary(x => x.SystemName, x => x.PeriodType);
       
    return nominalRawVariables.Select(rv => {
        var values = rv.Values.ToArray();
        var cdcf = new double[values.Length];
        periodTypeByAmountType.TryGetValue(rv.AmountType, out var period);

        if(period == PeriodType.BeginningOfPeriod)
        {
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = values[i] + cdcf.GetValidElement(i + 1) * yearlyDiscountRates.GetValidElement(i/12);
        }
        else
        { 
            for (var i = cdcf.Length - 1; i >= 0; i--)
                cdcf[i] = ( values[i] + cdcf.GetValidElement(i + 1) ) * yearlyDiscountRates.GetValidElement(i/12);
        }
        return rv with { Values = cdcf };
    })
    .ToDataCube();
}

# Import Configuration 

## Data Nodes

In [0]:
public GroupOfInsuranceContract ExtendGroupOfContract(GroupOfInsuranceContract gic, IDataRow datarow) => gic;

In [0]:
public GroupOfReinsuranceContract ExtendGroupOfContract(GroupOfReinsuranceContract gric, IDataRow datarow) => gric;

## Economic Basis Driver default

In [0]:
public static string GetDefaultEconomicBasisDriver(string valuationApproach, string liabilityType) {
    return (valuationApproach, liabilityType) switch {
        (ValuationApproaches.BBA, _) => EconomicBases.L,
        (ValuationApproaches.VFA, _) => EconomicBases.C,
        (ValuationApproaches.PAA, LiabilityTypes.LIC) => EconomicBases.C,
        _ => EconomicBases.N,
    };
}

## Interpolate

In [0]:
public static double[] Interpolate(this double[] cashflowValues, CashFlowPeriodicity periodicity, InterpolationMethod interpolationMethod)
{ 
    if (periodicity == CashFlowPeriodicity.Monthly)
        return cashflowValues;
    
    var frequency = periodicity switch {
        CashFlowPeriodicity.Yearly => 12,
        CashFlowPeriodicity.Quarterly => 4,
        _ => 1
        };
    
    return interpolationMethod switch {
        InterpolationMethod.Start        => cashflowValues.SelectMany(v => Enumerable.Range(0, frequency).Select(x => x == 0 ? v : default)).ToArray(),
        InterpolationMethod.Uniform or _ => cashflowValues.SelectMany(v => Enumerable.Range(0, frequency).Select( _ => v / (double)frequency)).ToArray()
    };

}

## Change Sign Rules

In [0]:
public static int GetSign(string importFormat, (string AocType, string AmountType, string EstimateType, bool IsReinsurance) variable, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache) => 1;

## Adjust Values for Cashflows

In [0]:
public static double[] AdjustValues(this double[] values, ImportArgs args, DataNodeData dataNodeData, int? AccidentYear) => values;

## EstimateType And AmountType

In [0]:
public static Dictionary<string, HashSet<string>> GetAmountTypesByEstimateType(Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache)
{
    return new Dictionary<string, HashSet<string>>(){
        {EstimateTypes.RA, new string[]{}.ToHashSet()},
        {EstimateTypes.C, new string[]{}.ToHashSet()},
        {EstimateTypes.L, new string[]{}.ToHashSet()},
        {EstimateTypes.LR, new string[]{}.ToHashSet()},
   };
}

In [0]:
public static (string, string) ParseAmountAndEstimateType (this IDataRow datarow, string format, 
    Dictionary<Type, Dictionary<string, string>> dimensionsWithExternalId, 
    Dictionary<string, EstimateType> estimateTypes, 
    Dictionary<string, AmountType> amountTypes) 
{
    return (datarow.Field<string>(nameof(RawVariable.AmountType)), datarow.Field<string>(nameof(RawVariable.EstimateType)));
}

## Non Attributable AmountTypes

In [0]:
public static HashSet<string> GetNonAttributableAmountTypes() => new string[]{AmountTypes.NE}.ToHashSet();

## Technical Margin EstimateType

In [0]:
public static HashSet<string> GetTechnicalMarginEstimateType()
{
    return new []{EstimateTypes.C, EstimateTypes.L, EstimateTypes.LR, }.ToHashSet();
}

## Import Actuals EstimateType

In [0]:
public static HashSet<string> GetImportActualEstimateType()
{
    return new []{EstimateTypes.A, EstimateTypes.AA, EstimateTypes.OA}.ToHashSet();
}

## Extend Parsed Variables

In [0]:
public static async Task<ActivityLog> ExtendParsedCoverageUnits (this IWorkspace workspace, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache, Systemorph.Vertex.Activities.ActivityVariable activity) {
    activity.Start();
    return activity.Finish();
}

In [0]:
public static async Task<ActivityLog> ExtendParsedVariables (this IWorkspace workspace, Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache, Systemorph.Vertex.Activities.IActivityVariable activity) =>
    await workspace.ExtendParsedCoverageUnits(hierarchyCache, activity);

# Helper variables used in calculation

In [0]:
public static class ComputationHelper{
    public static string [] AocTypeWithNoPv = new string[]{AocTypes.BOP, AocTypes.EA, AocTypes.AM, AocTypes.RCU};
} 

# Initialize the values given the Period

In [0]:
public static double[] SetProjectionValue(double value, int period = 0) => 
    period == 0 || Math.Abs(value) > Precision
        ? Enumerable.Repeat(0d, period + 1).Select((y, i) => i == period ? value : y).ToArray()
        : null; 